<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/Optimized_LDM_TXT2IM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latent Diffusion Models Text2Image

### https://arxiv.org/abs/2112.10752

### https://github.com/CompVis/latent-diffusion

Original Notebook by: [Eyal Gruss](https://eyalgruss.com) \([@eyaler](https://twitter.com/eyaler)\)

Optimizations by: [Aaron Gokaslan](https://twitter.com/SkyLi0n) and faster sampling by [RiverHasWings](https://twitter.com/rivershavewings)

# Note you need a GPU with 16GB of VRAM. If you get a K80, try again.

In [ ]:
!nvidia-smi

In [ ]:
#@title Setup
%cd /content
!git clone https://github.com/crowsonkb/latent-diffusion --depth 1
!git clone https://github.com/CompVis/taming-transformers --depth 1
!pip -q install -e ./taming-transformers
!pip -q install omegaconf pytorch-lightning torch-fidelity einops transformers
%cd latent-diffusion
!cp scripts/txt2img.py .
!mkdir -p models/ldm/text2img-large/
!wget -nc -O models/ldm/text2img-large/model.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt


In [ ]:
%%writefile txt2img.py
import argparse, os, sys, glob
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm.auto import tqdm, trange
from einops import rearrange
from torchvision.utils import make_grid

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cuda")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--prompt",
        type=str,
        nargs="?",
        default="a painting of a virus monster playing guitar",
        help="the prompt to render"
    )

    parser.add_argument(
        "--outdir",
        type=str,
        nargs="?",
        help="dir to write results to",
        default="outputs/txt2img-samples"
    )
    parser.add_argument(
        "--ddim_steps",
        type=int,
        default=200,
        help="number of ddim sampling steps",
    )

    parser.add_argument(
        "--plms",
        action='store_true',
        help="use plms sampling",
    )

    parser.add_argument(
        "--ddim_eta",
        type=float,
        default=0.0,
        help="ddim eta (eta=0.0 corresponds to deterministic sampling",
    )
    parser.add_argument(
        "--n_iter",
        type=int,
        default=1,
        help="sample this often",
    )

    parser.add_argument(
        "--H",
        type=int,
        default=256,
        help="image height, in pixel space",
    )

    parser.add_argument(
        "--W",
        type=int,
        default=256,
        help="image width, in pixel space",
    )

    parser.add_argument(
        "--n_samples",
        type=int,
        default=4,
        help="how many samples to produce for the given prompt",
    )

    parser.add_argument(
        "--scale",
        type=float,
        default=5.0,
        help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
    )
    opt = parser.parse_args()


    config = OmegaConf.load("configs/latent-diffusion/txt2img-1p4B-eval.yaml")  # TODO: Optionally download from same location as ckpt and chnage this logic
    model = load_model_from_config(config, "models/ldm/text2img-large/model.ckpt")  # TODO: check path

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)

    if opt.plms:
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir

    prompt = opt.prompt


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))

    all_samples=list()
    with torch.no_grad():
        with model.ema_scope():
            uc = None
            if opt.scale != 1.0:
                uc = model.get_learned_conditioning(opt.n_samples * [""])
            for n in trange(opt.n_iter, desc="Sampling"):
                c = model.get_learned_conditioning(opt.n_samples * [prompt])
                shape = [4, opt.H//8, opt.W//8]
                samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                 conditioning=c,
                                                 batch_size=opt.n_samples,
                                                 shape=shape,
                                                 verbose=False,
                                                 unconditional_guidance_scale=opt.scale,
                                                 unconditional_conditioning=uc,
                                                 eta=opt.ddim_eta)

                x_samples_ddim = model.decode_first_stage(samples_ddim)
                x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, min=0.0, max=1.0)

                for x_sample in x_samples_ddim:
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    Image.fromarray(x_sample.astype(np.uint8)).save(os.path.join(sample_path, f"{base_count:04}.png"))
                    base_count += 1
                all_samples.append(x_samples_ddim)


    # additionally, save as grid
    grid = torch.stack(all_samples, 0)
    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
    grid = make_grid(grid, nrow=opt.n_samples)

    # to image
    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
    Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{prompt.replace(" ", "-")}.png'))

    print(f"Your samples are ready and waiting four you here: \n{outpath} \nEnjoy.")


In [ ]:
%cd /content/latent-diffusion

#@title Generate
prompt = 'A sticker of Albert Einstein riding a hors' #@param {type: 'string'}
ddim_eta = 0 #@param {type: 'number'}
n_samples = 2 #@param {type: 'integer'}
n_iter = 4 #@param {type: 'integer'}
scale = 5 #@param {type: 'number'}
ddim_steps =  50#@param {type: 'integer'}
W = 256 #@param {type: 'integer'}
H = 256 #@param {type: 'integer'}
outdir = 'outputs' #@param {type: 'string'}
!mkdir -p $outdir
from google.colab.patches import cv2_imshow
import cv2
!python txt2img.py --prompt "$prompt" --ddim_eta $ddim_eta --n_samples $n_samples --n_iter $n_iter --scale $scale --ddim_steps $ddim_steps --H $H --W $W --outdir $outdir --plms
filename = f'{outdir}/{prompt.replace(" ", "-")}.png'
im = cv2.imread(filename)
cv2_imshow(im)

In [ ]:
#@title Download images
!zip -jrqFS ldm.zip "$outdir"
from google.colab import files
files.download('ldm.zip')